# Analyze Hammerdb Logs

In [ ]:
# S3_LOGS_URLS = ['412URL', '413URL'] - for comparing 2 artifacts 
S3_LOGS_URLS = ['', '']
# Compare versions/ database
versions = ['4.12', '4.13']
database = ['PostgreSQL', 'PostgreSQL']

# Prerequsite

In [ ]:

# SET log level
# logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)
logging.info("Prerequisite")

# extract credentials
import configparser
config = configparser.ConfigParser()
config.read('logs.conf')
username = config.get("data", 'user') 
password = config.get("data", 'password')


# Update fedora
!dnf update -y

from benchmark_runner.jupyterlab.templates.logs_operations.logs_operations import LogsOperations
from benchmark_runner.jupyterlab.templates.analyze_workloads.analyze_hammerdb_logs.analyze_hammerdb_logs import AnalyzeHammerdbLogs

logs_operations_obj = []
analyze_hammerdb_logs_obj = []

#Logs operations
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    logs_operations = LogsOperations(s3_logs_url=S3_LOGS_URL)
    logs_operations_obj.append(logs_operations)
    
# Logs cleanup
logs_operations_obj[0].cleanup()

for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_hammerdb_logs = AnalyzeHammerdbLogs(s3_logs_url=S3_LOGS_URL)
    analyze_hammerdb_logs_obj.append(analyze_hammerdb_logs)

# Output clean output display
analyze_hammerdb_logs_obj[0].clear_output()


## Download S3 logs

In [ ]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    logs_operations_obj[i].download_s3_logs(username=username, password=password)

## Untar logs & chmod 

In [ ]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    logs_operations_obj[i].untar_and_chmod_logs()

# Get Hammerdb log file

In [ ]:
workload_log_files=[]
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    workload_log_files.append(analyze_hammerdb_logs_obj[i].get_hammerdb_log_file())

# Extract Hammerdb result

In [ ]:
hammerdb_results=[]
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    hammerdb_results.append(analyze_hammerdb_logs_obj[i].extract_hammerdb_result(workload_log_file=workload_log_files[i]))

# Display Hammerdb result

In [ ]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_hammerdb_logs_obj[i].display_hammerdb_result(hammerdb_result=hammerdb_results[i], workload_log_file=workload_log_files[i])

# Comapare results

In [ ]:
if len(S3_LOGS_URLS)>1:
    analyze_hammerdb_logs_obj[0].compare_results(result1=hammerdb_results[0], result2=hammerdb_results[1], legend_label1=versions[0], legend_label2=versions[1], database=database[0])